Uma versão mais completa do blockchain, com possibilidade de PoW e mineração.

Vamos começar criando uma classe pro novo blockchain

In [1]:
from hashlib import sha256
import json

In [3]:
class Block:
    
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce
        
    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

JSON (JavaScript Object Notation) é um formato popular de representação de dados.

O comando json.dumps() converte um objeto em Python para uma string.

Se sort_keys (padrão False) for true o output do dicionário será organizado por key.

Precisamos criar a cadeia de blocos que será o block chain.

In [5]:
import time

class Blockchain:
    
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()
        
    def create_genesis_block(self):
        
        genesis_block = Block(0,[],time.time(),"0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
        
    @property
    def last_block(self):
        return self.chain[-1]

Teste rápido:

In [7]:
Lancoin = Blockchain()

In [8]:
Lancoin.__dict__

{'unconfirmed_transactions': [], 'chain': [<__main__.Block at 0x7f819c29a610>]}

In [12]:
Lancoin.chain[0].__dict__

{'index': 0,
 'transactions': [],
 'timestamp': 1678876800.3912663,
 'previous_hash': '0',
 'nonce': 0,
 'hash': 'd18cf6f2c8e129c98564dc5d2d21fe5bcf454bb9e16364399a9c0515122f0170'}

PoW basciamente significa encontrar um número que faça com que o hash do bloco começe com um número predeterminado de zeros. Esse número que mudamos é o chamado nonce.

In [30]:
import time

class Blockchain:
    
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()
        
    def create_genesis_block(self):
        
        genesis_block = Block(0,[],time.time(),"0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
        
        
    difficulty = 2
    
    def proof_of_work(self,block):
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0'*Blockchain.difficulty):
            
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
    
    
        
    @property
    def last_block(self):
        return self.chain[-1]

In [31]:
Lancoin = Blockchain()
Lancoin.__dict__

{'unconfirmed_transactions': [], 'chain': [<__main__.Block at 0x7f819c0c3730>]}

In [32]:
Lancoin.chain[0].__dict__

{'index': 0,
 'transactions': [],
 'timestamp': 1678877580.9420972,
 'previous_hash': '0',
 'nonce': 0,
 'hash': '1dc2cfa6e6fe5da958d2d13e715f06c275c7b0b6e94e7220cc19ced78d41106b'}

In [33]:
Lancoin.proof_of_work(Lancoin.chain[0])

'0000087ca01ae8910e07e738fd186ed5a001cfcd3ce21f5929de7a57f94e87f2'

In [34]:
import time

class Blockchain:
    
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()
        
    def create_genesis_block(self):
        
        genesis_block = Block(0,[],time.time(),"0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
        
        
    difficulty = 2
    
    def proof_of_work(self,block):
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0'*Blockchain.difficulty):
            
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
    
    #parte nova
    
    def add_block(self,block,proof):
        previous_hash = self.last_block.hash
        
        if previous_hash != block.previous_hash:
            return False
        
        if not self.is_valid_proof(block,proof):
            return False
        block.hash = proof
        self.chain.append(block)
    
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0'*Blockchain.difficulty) and block_hash == block.compute_hash())
    
    def add_new_transaction(self,transaction):
        self.unconfirmed_transactions.append(transaction)
        
        
    def mine(self):
        if not self.unconfirmed_transactions:
            return False
        
        last_block = self.last_block
        
        new_block = Block(index=last_block.index+1,
                         transactions=self.unconfirmed_transactions,
                         timestamp=time.time(),
                         previous_hash=last_block.hash)
        
        proof = self.proof_of_work(new_block)
        self.add_block(new_block,proof)
        self.unconfirmed_transactions = []
        return new_block.index
    
    #parte nova
    
    
        
    @property
    def last_block(self):
        return self.chain[-1]

Hora da verdade. Vamos testar.

In [35]:
lancoin = Blockchain()

In [37]:
print(lancoin.__dict__)
print(lancoin.chain[0].__dict__)

{'unconfirmed_transactions': [], 'chain': [<__main__.Block object at 0x7f819c21db20>]}
{'index': 0, 'transactions': [], 'timestamp': 1678878778.3157554, 'previous_hash': '0', 'nonce': 0, 'hash': 'fd319b43ac4749bb5a195c443488e6efb89d392e98007bcd8b9a9748c59e43c8'}


In [38]:
lancoin.difficulty

2

In [39]:
lancoin.proof_of_work(lancoin.chain[0])

'000d0c040ec9c4f7a90d69181ec7670cb6fdbb88a170d43f82e219045e3f2dc4'

In [40]:
print(lancoin.chain[0].__dict__)

{'index': 0, 'transactions': [], 'timestamp': 1678878778.3157554, 'previous_hash': '0', 'nonce': 38, 'hash': 'fd319b43ac4749bb5a195c443488e6efb89d392e98007bcd8b9a9748c59e43c8'}


In [41]:
lancoin.chain[0].compute_hash()

'000d0c040ec9c4f7a90d69181ec7670cb6fdbb88a170d43f82e219045e3f2dc4'

Vamos adicionar transações:

In [42]:
lancoin.add_new_transaction("t1")

In [43]:
lancoin.add_new_transaction("t2")

In [44]:
lancoin.add_new_transaction("t3")

In [46]:
lancoin.unconfirmed_transactions

['t1', 't2', 't3']

In [47]:
lancoin.mine()

1

In [48]:
print(lancoin.__dict__)

{'unconfirmed_transactions': [], 'chain': [<__main__.Block object at 0x7f819c21db20>, <__main__.Block object at 0x7f819c1debe0>]}


Agora a chain tem dois elementos. Vamos ver os dois:

In [49]:
print(lancoin.chain[0].__dict__)
print(lancoin.chain[1].__dict__)

{'index': 0, 'transactions': [], 'timestamp': 1678878778.3157554, 'previous_hash': '0', 'nonce': 38, 'hash': 'fd319b43ac4749bb5a195c443488e6efb89d392e98007bcd8b9a9748c59e43c8'}
{'index': 1, 'transactions': ['t1', 't2', 't3'], 'timestamp': 1678879153.198021, 'previous_hash': 'fd319b43ac4749bb5a195c443488e6efb89d392e98007bcd8b9a9748c59e43c8', 'nonce': 388, 'hash': '0086690dcc643b13e5a8f446ab660314e4949c2b65a81c3aecb9ba4d2c53b046'}


In [50]:
lancoin.unconfirmed_transactions

[]

In [51]:
lancoin.mine()

False

In [52]:
lancoin.add_new_transaction("t4")
lancoin.add_new_transaction("t5")
lancoin.add_new_transaction("t6")

In [54]:
lancoin.mine()

2

In [55]:
print(lancoin.__dict__)
print(lancoin.chain[0].__dict__)
print(lancoin.chain[1].__dict__)
print(lancoin.chain[2].__dict__)

{'unconfirmed_transactions': [], 'chain': [<__main__.Block object at 0x7f819c21db20>, <__main__.Block object at 0x7f819c1debe0>, <__main__.Block object at 0x7f819c1def70>]}
{'index': 0, 'transactions': [], 'timestamp': 1678878778.3157554, 'previous_hash': '0', 'nonce': 38, 'hash': 'fd319b43ac4749bb5a195c443488e6efb89d392e98007bcd8b9a9748c59e43c8'}
{'index': 1, 'transactions': ['t1', 't2', 't3'], 'timestamp': 1678879153.198021, 'previous_hash': 'fd319b43ac4749bb5a195c443488e6efb89d392e98007bcd8b9a9748c59e43c8', 'nonce': 388, 'hash': '0086690dcc643b13e5a8f446ab660314e4949c2b65a81c3aecb9ba4d2c53b046'}
{'index': 2, 'transactions': ['t4', 't5', 't6'], 'timestamp': 1678879322.095143, 'previous_hash': '0086690dcc643b13e5a8f446ab660314e4949c2b65a81c3aecb9ba4d2c53b046', 'nonce': 257, 'hash': '0024e5c46bf8a85e369ece9db401d6a40645a219809bf974d2e709473aa460cb'}
